# Задание

Вариант 1: Разработать программу, выполняющую кластеризацию пороговым методом. Для вычисления расстояний между образами необходимо использовать формулу расстояния Евклида и расстояния Канберра.

# Реализация

Зададим множество точек (образов) для дальнейшей кластеризации.

In [1]:
import numpy as np

data = np.array([
    (0,1,1), (0,1,7), (5,7,4), (0,5,5), (9,4,5), (7,1,2), (10,0,19),
    (0,12,7), (-5,-4,5), (20,10,15), (0,16,-16), (-1,9,-30),
    (18,0,17), (6,18,4)
], dtype='float32')

data

array([[  0.,   1.,   1.],
       [  0.,   1.,   7.],
       [  5.,   7.,   4.],
       [  0.,   5.,   5.],
       [  9.,   4.,   5.],
       [  7.,   1.,   2.],
       [ 10.,   0.,  19.],
       [  0.,  12.,   7.],
       [ -5.,  -4.,   5.],
       [ 20.,  10.,  15.],
       [  0.,  16., -16.],
       [ -1.,   9., -30.],
       [ 18.,   0.,  17.],
       [  6.,  18.,   4.]], dtype=float32)

Функции для расчета расстояния между точками. Используется расстояние Евклида:

$$ \sum_{i=1}^{n} (x_i - y_i)^2. $$

И расстояние Канберра:

$$ \sum_{i=1}^{n} \left|\dfrac{x_i - y_i}{\left| x_i \right| + \left| y_i \right|}\right| $$

При вычислении расстояния Канберра возможно деление на ноль, поэтому все нули в знаменателе заменяются на $10^{-5}$.

In [2]:
def euclidian_distances(x, y, p=2):
    return np.sum((x - y)**p, axis=1)

def canberra_distances(x, y):
    divisor = np.abs(x)+np.abs(y)
    np.place(divisor, (divisor == 0), 1e-5)
    return np.sum(np.abs((x-y)/divisor), axis=1)

Реализуем функцию кластеризации в соответствии выбранным методом и пару вспомогательных функций

In [3]:
def calculate_distances(func, image, clusters):
    """Вспомогательная функция для вычисления расстояния между заданным образом и кластерами"""
    image_t = np.tile(image, (clusters.shape[0], 1))
    distance = func(image_t, clusters)
    return distance

def calculate_cluster_center(cluster):
    """Вычисляет арифмитическое среднее"""
    return np.sum(cluster, axis=0)/cluster.shape[0]

def clusterize(data, threshold, func):
    """Выполняет кластеризацию данных, используя заданный порог и функцию вычисления расстояния"""
    clusters_centers = data[:1].copy()
    clusters_images = [data[:1].copy()]
    for image in data[1:]:
        distances = calculate_distances(func, image, clusters_centers)
        if np.all(distances > threshold):
            # новый кластер
            clusters_centers = np.append(clusters_centers, np.array([image.copy()]), axis=0)
            clusters_images.append(np.array([image.copy()]))
        else:
            # добавляю в существующий кластер
            idx = np.argmin(distances)
            newcluster = np.append(clusters_images[idx], np.array([image.copy()]), axis=0)
            clusters_images[idx] = newcluster
            clusters_centers[idx] = calculate_cluster_center(newcluster)
    # форматирование данных
    number_of_clusters = len(clusters_centers)
    result = np.zeros((number_of_clusters, 2), dtype='O')
    for i in range(number_of_clusters):
        result[i,0] = clusters_centers[i]
        result[i,1] = clusters_images[i]
    return result

Теперь можно поэксперементировать с кластеризацией, выбирая предел и функцию расстояния.

In [4]:
import pandas as pd
from ipywidgets import interact, interactive, widgets
from IPython.display import display

pd.set_option('max_colwidth', 300)

distance_functions = {'Расстояние Евклида': euclidian_distances, 'Расстояние Канберра': canberra_distances}

threshold_sl = widgets.FloatSlider(min=0., max=5., value=2.)
max_threshold_sl = widgets.FloatSlider(min=0.5, max=300., value=5., description='max threshold')
max_threshold_sl.observe(lambda nx: setattr(threshold_sl, 'max', max_threshold_sl.value))
display(max_threshold_sl)
@interact(threshold=threshold_sl, func=distance_functions)
def interactive_clusterize(threshold, func):
    clusterized_data = clusterize(data, threshold, func)
    return pd.DataFrame(clusterized_data, index=np.arange(clusterized_data.shape[0]), columns=['Центр кластера', 'Образы'])

FloatSlider(value=5.0, description='max threshold', max=300.0, min=0.5)

interactive(children=(FloatSlider(value=2.0, description='threshold', max=5.0), Dropdown(description='func', options={'Расстояние Евклида': <function euclidian_distances at 0x7f02987797b8>, 'Расстояние Канберра': <function canberra_distances at 0x7f0298779730>}, value=<function euclidian_distances at 0x7f02987797b8>), Output()), _dom_classes=('widget-interact',))